<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/TITANIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt


In [4]:
test = pd.read_csv('/content/test.csv')
test.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [7]:
train = pd.read_csv('/content/train.csv')
train.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [19]:
lista_spec = ['PassengerId', 'Survived','Ticket','Name']

train.drop(lista_spec,axis=1,inplace=True)


In [20]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(train)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,Pclass,int64,0,0.00,3
1,SibSp,int64,0,0.00,7
2,Parch,int64,0,0.00,7
3,Age,float64,177,19.87,88
4,Fare,float64,0,0.00,248
5,Sex,object,0,0.00,2
6,Cabin,object,687,77.10,147
7,Embarked,object,2,0.22,3


In [21]:
# Variáveis numéricas
# Filtrando o DataFrame
missing_cutoff = 70
df_numericas_drop_nulos = metadados[(metadados['percent_nulos'] >= missing_cutoff)]
lista_drop_nulos_numericas = list(df_numericas_drop_nulos.nome_variavel.values)
lista_drop_nulos_numericas

['Cabin']

In [22]:
# retirando lista de variáveis com alto percentual de nulos
train2 = train.drop(axis=1,columns=lista_drop_nulos_numericas)
train2.shape

(891, 7)

In [23]:
# substituindo nulos pela média da variável numérica
train2 = train2.fillna(train2.select_dtypes(include=['float64', 'int64','int32']).mean())
train2.shape

(891, 7)

In [24]:
# substituindo nulos por "Desconhecido" para categóricas
categorical_cols = train2.select_dtypes(include=['object']).columns
train2[categorical_cols] = train2[categorical_cols].fillna('Desconhecido')
train2.shape

(891, 7)

In [25]:
metadados = generate_metadata(train2)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,Pclass,int64,0,0.0,3
1,SibSp,int64,0,0.0,7
2,Parch,int64,0,0.0,7
3,Age,float64,0,0.0,89
4,Fare,float64,0,0.0,248
5,Sex,object,0,0.0,2
6,Embarked,object,0,0.0,4


In [26]:
from sklearn.preprocessing import MinMaxScaler

# Instanciando o scaler
scaler = MinMaxScaler()

# Selecionando colunas numéricas
numeric_cols = train2.select_dtypes(include=['float64', 'int64','int32']).columns

# Aplicando a normalização
train2[numeric_cols] = scaler.fit_transform(train2[numeric_cols])
train2.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,male,0.271174,0.125,0.0,0.014151,S
1,0.0,female,0.472229,0.125,0.0,0.139136,C
2,1.0,female,0.321438,0.000,0.0,0.015469,S
3,0.0,female,0.434531,0.125,0.0,0.103644,S
4,1.0,male,0.434531,0.000,0.0,0.015713,S


In [28]:
from sklearn.preprocessing import OneHotEncoder
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

card_cutoff = 2
df_categ_onehot = metadados[(metadados['cardinalidade'] <= card_cutoff) & (metadados['tipo'] == 'object')]
lista_onehot = list(df_categ_onehot.nome_variavel.values)
print('Lista de vars para OneHot Encoding: ',lista_onehot)

# Instanciando o encoder
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Aplicando o one-hot encoding
encoded_data = encoder.fit_transform(train2[lista_onehot])

# Criando um DataFrame com as colunas codificadas, mantendo o índice original
encoded_cols = encoder.get_feature_names_out(lista_onehot)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=train2.index)

# Concatenando o DataFrame codificado com o DataFrame original
train3 = pd.concat([train2.drop(lista_onehot, axis=1), encoded_df], axis=1)

train3.head()

Lista de vars para OneHot Encoding:  ['Sex']


,Pclass,Age,SibSp,Parch,Fare,Embarked,Sex_male
0,1.0,0.271174,0.125,0.0,0.014151,S,1.0
1,0.0,0.472229,0.125,0.0,0.139136,C,0.0
2,1.0,0.321438,0.000,0.0,0.015469,S,0.0
3,0.0,0.434531,0.125,0.0,0.103644,S,0.0
4,1.0,0.434531,0.000,0.0,0.015713,S,1.0


In [33]:
card_cutoff = 2
df_categ_labelenc = metadados[(metadados['cardinalidade'] > card_cutoff) & (metadados['tipo'] == 'object')]
lista_lenc = list(df_categ_labelenc.nome_variavel.values)
print('Lista de vars para Label Encoding: ',lista_lenc)


Lista de vars para Label Encoding:  ['Embarked']


In [34]:
from sklearn.preprocessing import LabelEncoder

# Aplicando LabelEncoder nas colunas desejadas
for col in lista_lenc:
    encoder = LabelEncoder()
    train3[col] = encoder.fit_transform(train3[col])

train3.head()


,Pclass,Age,SibSp,Parch,Fare,Embarked,Sex_male
0,1.0,0.271174,0.125,0.0,0.014151,3,1.0
1,0.0,0.472229,0.125,0.0,0.139136,0,0.0
2,1.0,0.321438,0.000,0.0,0.015469,3,0.0
3,0.0,0.434531,0.125,0.0,0.103644,3,0.0
4,1.0,0.434531,0.000,0.0,0.015713,3,1.0
